In [8]:
import argparse, time, math
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.data import register_data_args
#from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset


def gcn_msg(edge):
    msg = edge.src['h'] * edge.src['norm']
    return {'m': msg}


def gcn_reduce(node):
    accum = torch.sum(node.mailbox['m'], 1) * node.data['norm']
    return {'h': accum}


class NodeApplyModule(nn.Module):
    def __init__(self, out_feats, activation=torch.sigmoid, bias=True):
        super(NodeApplyModule, self).__init__()
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_feats))
        else:
            self.bias = None
        self.activation = activation
        self.reset_parameters()

    def reset_parameters(self):
        if self.bias is not None:
            stdv = 1. / math.sqrt(self.bias.size(0))
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, nodes):
        h = nodes.data['h']
        if self.bias is not None:
            h = h + self.bias
        if self.activation:
            h = self.activation(h)
        return {'h': h}


class GCNLayer(nn.Module):
    def __init__(self,
                 g,
                 in_feats,
                 out_feats,
                 activation,
                 dropout,
                 bias=True):
        super(GCNLayer, self).__init__()
        self.g = g
        self.weight = nn.Parameter(torch.Tensor(in_feats, out_feats))
        if dropout:
            self.dropout = nn.Dropout(p=dropout)
        else:
            self.dropout = 0.
        self.node_update = NodeApplyModule(out_feats, activation, bias)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)

    def forward(self, h):
        if self.dropout:
            h = self.dropout(h)
        self.g.ndata['h'] = torch.mm(h, self.weight)
        self.g.update_all(gcn_msg, gcn_reduce, self.node_update)
        h = self.g.ndata.pop('h')
        return h

class GCN(nn.Module):
    def __init__(self,
                 g,
                 in_feats,
                 n_hidden,
                 n_classes,
                 n_layers,
                 activation,
                 dropout):
        super(GCN, self).__init__()
        self.layers = nn.ModuleList()
        # input layer
        self.layers.append(GCNLayer(g, in_feats, n_hidden, activation, dropout))
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(GCNLayer(g, n_hidden, n_hidden, activation, dropout))
        # output layer
        self.layers.append(GCNLayer(g, n_hidden, n_classes, None, dropout))

        #
    def forward(self, features):
        h = features
#         for i in range(len(self.layers)-1):
#             layer = self.layers[i]
#             h = layer(h)
#             if i == 2:
#                 print(h.shape)
#                 np.savetxt('gcn_5_0.txt',h.detach().numpy(),delimiter=' ', newline='\n',)
        for layer in self.layers:
            h = layer(h)
        return h

In [9]:
def evaluate(model, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        probas = F.softmax(logits)
        
#         correct = torch.sum(indices == labels)
#         return correct.item() * 1.0 / len(labels)
        return metrics(labels.cpu().detach(), indices.cpu().detach(), probas.cpu().detach()[:,1])

In [10]:
def metrics(y_true, y_pred, y_prob):

    y_true, y_pred, y_prob = y_true.numpy(), y_pred.numpy(), y_prob.numpy()
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    pos_acc = tp / sum(y_true)
    neg_acc = tn / (len(y_pred) - sum(y_pred)) # [y_true=0 & y_pred=0] / y_pred=0
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    recall = tp / (tp+fn)
    precision = tp / (tp+fp)
    f1 = 2*precision*recall / (precision+recall)
    
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    f = open('result4.txt','a', encoding='utf-8')
    f.writelines('tn = {}, fp = {}, fn = {}, tp = {} \n'.format(tn, fp, fn, tp))
    f.writelines('y_pred: 0 = {} | 1 = {}\n'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
    f.writelines('y_true: 0 = {} | 1 = {}\n'.format(Counter(y_true)[0], Counter(y_true)[1]))
    f.writelines('acc={:.4f}|precision={:.4f}|recall={:.4f}|f1={:.4f}|auc={:.4f}|aupr={:.4f}|pos_acc={:.4f}|neg_acc={:.4f}\n'.format(accuracy, precision, recall, f1, roc_auc, aupr, pos_acc, neg_acc))
    f.close()
#     np.savetxt("result.txt",'tn = {}, fp = {}, fn = {}, tp = {} \n'.format(tn, fp, fn, tp))
#     np.savetxt("result.txt",'y_pred: 0 = {} | 1 = {}\n'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
#     np.savetxt("result.txt",'y_true: 0 = {} | 1 = {}\n'.format(Counter(y_true)[0], Counter(y_true)[1]))
#     np.savetxt("result.txt",'acc={:.4f}|precision={:.4f}|recall={:.4f}|f1={:.4f}|auc={:.4f}|aupr={:.4f}|pos_acc={:.4f}|neg_acc={:.4f}\n'.format(accuracy, precision, recall, f1, roc_auc, aupr, pos_acc, neg_acc))
    return (y_true, y_pred, y_prob), (accuracy, precision, recall, f1, roc_auc, aupr, pos_acc, neg_acc)

In [11]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import scipy.sparse as sp
from copy import deepcopy
import warnings 
import os
# from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
import json
warnings.filterwarnings("ignore") 

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from collections import Counter
from sklearn.model_selection import KFold
import argparse

In [12]:
def main(args, g, features, labels, train_idx):

    if args.gpu >= 0:
        cuda = True
        device = torch.device('cuda:%d' % args.gpu)
    else:
        cuda = False
        device = torch.device('cpu')
    
    num_nodes = g.number_of_nodes()
    train_mask = np.zeros(num_nodes, dtype = 'int64')
    train_mask[train_idx] = 1
    test_mask = 1 - train_mask
    print(Counter(train_mask), Counter(test_mask))
    train_mask = torch.BoolTensor(train_mask)
    test_mask = torch.BoolTensor(test_mask)

    g.ndata['feat'] = features
    g.ndata['label'] = labels
    g.ndata['train_mask'] = train_mask
    g.ndata['test_mask'] = test_mask
    
    g = g.to(device)
        
    in_feats = features.shape[1]
    n_classes = 2
    n_edges = g.number_of_edges()

    features, labels = features.to(device), labels.to(device)
    
    print("""----Data statistics------'
      #Edges %d
      #Classes %d
      #Train samples %d
      #Test samples %d""" %
          (n_edges, n_classes,
          train_mask.int().sum().item(),
          test_mask.int().sum().item()))

    # normalization
    degs = g.in_degrees().float()
    norm = torch.pow(degs, -0.5)
    norm[torch.isinf(norm)] = 0
    if cuda:
        norm = norm.cuda()
    g.ndata['norm'] = norm.unsqueeze(1)

    # create GCN model
    model = GCN(g,
                in_feats,
                args.n_hidden,
                n_classes,
                args.n_layers,
                F.relu,
                args.dropout)

    if cuda:
        model.cuda()
    loss_fcn = torch.nn.CrossEntropyLoss()

    # use optimizer
    optimizer = torch.optim.Adamax(model.parameters(),
                                 lr=args.lr,
                                 weight_decay=args.weight_decay)

    # initialize graph
    dur = []
    for epoch in range(args.n_epochs):
        model.train()

        t0 = time.time()
        # forward
        logits = model(features)
        loss = loss_fcn(logits[train_mask], labels[train_mask])
        loss.requires_grad_(True)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        dur.append(time.time() - t0)
        
        print('=====Epoch {} | Time(s) {:.4f} | Loss {:.4f} | ETputs(KTEPS) {:.2f}'.format(epoch, np.mean(dur), loss.item(), n_edges / np.mean(dur) / 1000))
        
        ys_train, metrics_train = evaluate(model, features, labels, train_mask)
        ys_test, metrics_test = evaluate(model, features, labels, test_mask)
        
    return ys_train, metrics_train, ys_test, metrics_test

In [13]:
def run(args, task, isbalance, balance, n_neigh):
    pwd = r'D:/小麦/MDA-GCNFTG-main/MDA-GCNFTG-main/0_data/'
    
    if isbalance:
        node_feature_label = pd.read_csv(pwd + 'node_feature_label_.csv', index_col = 0)
    
    train_test_id_idx = np.load(r'D:/小麦/MDA-GCNFTG-main/MDA-GCNFTG-main/data/task_' + task + balance + '__testlabel0_knn_edge_train_test_index_all.npz', allow_pickle = True)
    train_index_all = train_test_id_idx['train_index_all']
    test_index_all = train_test_id_idx['test_index_all']
#     绘图
    train_idx = train_index_all[0]
    test_idx = train_index_all[0]
    
    num_nodes = node_feature_label.shape[0]
    features = torch.FloatTensor(np.array(node_feature_label.iloc[:, 3:]))
    labels = torch.LongTensor(np.array(node_feature_label['label']))
    
    fold = 4
    #for train_idx, test_idx in zip(train_index_all, test_index_all):
        
    print('=====Fold {}============================================='.format(fold))
    f = open('result4.txt','a', encoding='utf-8')
    f.writelines('=====Fold {}=============================================\n'.format(fold))
    f.close()
    knn_graph_file = 'task_' + task + balance + '__testlabel0_knn' + str(n_neigh) + 'neighbors_edge__fold' + str(fold) + '.npz'
    knn_neighbors_graph = sp.load_npz(pwd + knn_graph_file)

    edge_src = knn_neighbors_graph.nonzero()[0]
    edge_dst = knn_neighbors_graph.nonzero()[1]

    g = dgl.DGLGraph()
    g.add_nodes(num_nodes)
    g.add_edges(edge_src, edge_dst)
    g = dgl.add_self_loop(g)
    print(g)

    ys_train, metrics_train, ys_test, metrics_test = main(args, g, features, labels, train_idx)
#         print(metrics_test)
        #fold += 1
    
    return node_feature_label, train_index_all, test_index_all, knn_neighbors_graph, g, ys_train, metrics_train, ys_test, metrics_test

# RUN

In [14]:
# balance data
if __name__ == '__main__':
    
    parser = argparse.ArgumentParser(description='GCN')
    register_data_args(parser)
    parser.add_argument("--dropout", type=float, default=0.3,
            help="dropout probability")
    parser.add_argument("--gpu", type=int, default=-1,
            help="gpu")
    parser.add_argument("--lr", type=float, default=0.003,
            help="learning rate")
    #一开始是500，画图设置为250
    parser.add_argument("--n-epochs", type=int, default=250,
            help="number of training epochs")
    parser.add_argument("--n-hidden", type=int, default=512,
            help="number of hidden gcn units")
    parser.add_argument("--n-layers", type=int, default=3,
            help="number of hidden gcn layers")
    parser.add_argument("--weight-decay", type=float, default=5e-4,
            help="Weight for L2 loss")
    args = parser.parse_args(args = [])
    print(args)
    
    for isbalance in [True]:
        
        if isbalance:
            balance = ''
        else:
            balance = '__nobalance'
        #tp 7 的第三折断的   
        for task in ['Tpe']:
            
            for n_neigh in [10]:
                
                print('************** isbalance = {} | task = {} | n_neigh = {}'.format(isbalance, task, n_neigh))
                f = open('result4.txt','a', encoding='utf-8')
                f.writelines('************** isbalance = {} | task = {} | n_neigh = {}\n'.format(isbalance, task, n_neigh))
                f.close()
                node_feature_label, train_index_all, test_index_all, \
                knn_neighbors_graph, g, ys_train, metrics_train, ys_test, metrics_test = run(args,
                                                                                              task, 
                                                                                              isbalance, 
                                                                                              balance, 
                                                                                              n_neigh)

Namespace(dataset=None, dropout=0.3, gpu=-1, lr=0.003, n_epochs=250, n_hidden=512, n_layers=3, weight_decay=0.0005)
************** isbalance = True | task = Tpe | n_neigh = 10
=====Fold 4=============================================
Graph(num_nodes=47116, num_edges=518276,
      ndata_schemes={}
      edata_schemes={})
Counter({1: 41104, 0: 6012}) Counter({0: 41104, 1: 6012})
----Data statistics------'
      #Edges 518276
      #Classes 2
      #Train samples 41104
      #Test samples 6012
=====Epoch 0 | Time(s) 63.8850 | Loss 1.0194 | ETputs(KTEPS) 8.11
=====Epoch 1 | Time(s) 61.5610 | Loss 31.4834 | ETputs(KTEPS) 8.42
=====Epoch 2 | Time(s) 60.0910 | Loss 11.7897 | ETputs(KTEPS) 8.62
=====Epoch 3 | Time(s) 59.5478 | Loss 3.2497 | ETputs(KTEPS) 8.70
=====Epoch 4 | Time(s) 59.1316 | Loss 1.0843 | ETputs(KTEPS) 8.76
=====Epoch 5 | Time(s) 59.0438 | Loss 0.7150 | ETputs(KTEPS) 8.78
=====Epoch 6 | Time(s) 59.3420 | Loss 0.7163 | ETputs(KTEPS) 8.73
=====Epoch 7 | Time(s) 59.5242 | Loss 0.7

=====Epoch 115 | Time(s) 68.5172 | Loss 0.6343 | ETputs(KTEPS) 7.56
=====Epoch 116 | Time(s) 68.5241 | Loss 0.6321 | ETputs(KTEPS) 7.56
=====Epoch 117 | Time(s) 68.5549 | Loss 0.6332 | ETputs(KTEPS) 7.56
=====Epoch 118 | Time(s) 68.5326 | Loss 0.6300 | ETputs(KTEPS) 7.56
=====Epoch 119 | Time(s) 68.4916 | Loss 0.6310 | ETputs(KTEPS) 7.57
=====Epoch 120 | Time(s) 68.4539 | Loss 0.6315 | ETputs(KTEPS) 7.57
=====Epoch 121 | Time(s) 68.4118 | Loss 0.6297 | ETputs(KTEPS) 7.58
=====Epoch 122 | Time(s) 68.3701 | Loss 0.6281 | ETputs(KTEPS) 7.58
=====Epoch 123 | Time(s) 68.3276 | Loss 0.6246 | ETputs(KTEPS) 7.59
=====Epoch 124 | Time(s) 68.2900 | Loss 0.6294 | ETputs(KTEPS) 7.59
=====Epoch 125 | Time(s) 68.2851 | Loss 0.6245 | ETputs(KTEPS) 7.59
=====Epoch 126 | Time(s) 68.2865 | Loss 0.6270 | ETputs(KTEPS) 7.59
=====Epoch 127 | Time(s) 68.2919 | Loss 0.6234 | ETputs(KTEPS) 7.59
=====Epoch 128 | Time(s) 68.2650 | Loss 0.6247 | ETputs(KTEPS) 7.59
=====Epoch 129 | Time(s) 68.2297 | Loss 0.6235 |

=====Epoch 236 | Time(s) 69.6540 | Loss 0.5753 | ETputs(KTEPS) 7.44
=====Epoch 237 | Time(s) 69.6490 | Loss 0.5738 | ETputs(KTEPS) 7.44
=====Epoch 238 | Time(s) 69.6256 | Loss 0.5733 | ETputs(KTEPS) 7.44
=====Epoch 239 | Time(s) 69.6150 | Loss 0.5733 | ETputs(KTEPS) 7.44
=====Epoch 240 | Time(s) 69.6270 | Loss 0.5726 | ETputs(KTEPS) 7.44
=====Epoch 241 | Time(s) 69.6382 | Loss 0.5782 | ETputs(KTEPS) 7.44
=====Epoch 242 | Time(s) 69.6405 | Loss 0.5753 | ETputs(KTEPS) 7.44
=====Epoch 243 | Time(s) 69.6780 | Loss 0.5747 | ETputs(KTEPS) 7.44
=====Epoch 244 | Time(s) 69.6594 | Loss 0.5742 | ETputs(KTEPS) 7.44
=====Epoch 245 | Time(s) 69.6410 | Loss 0.5749 | ETputs(KTEPS) 7.44
=====Epoch 246 | Time(s) 69.6253 | Loss 0.5737 | ETputs(KTEPS) 7.44
=====Epoch 247 | Time(s) 69.6636 | Loss 0.5750 | ETputs(KTEPS) 7.44
=====Epoch 248 | Time(s) 69.7438 | Loss 0.5736 | ETputs(KTEPS) 7.43
=====Epoch 249 | Time(s) 69.7826 | Loss 0.5700 | ETputs(KTEPS) 7.43
